# Create Slice for Fabric Rack
<br>This is the slice creation for the ```TCP Traffic``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the TCP Traffic Assignment.
### 1.1 Reserve Resources
In this section we will declare the structure of the slice we will be using.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "TCP_Traffic_intro"
site = "EDUKY"
print(site)


nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10

try:
    # Create a slice
    slice = fablib.new_slice(name=slice_name)
    #first Node
    ND1 = slice.add_node(name="ND_A", site=site)
    ND1.set_capacities(cores=cores, ram=ram, disk=disk)
    ND1.set_image(image)
    Net_ND1 = ND1.add_component(model=nicmodel, name="ND_A").get_interfaces()[0]
    

    #Second Node
    ND2 = slice.add_node(name="ND_B", site=site)
    ND2.set_capacities(cores=cores, ram=ram, disk=disk)
    ND2.set_image(image)
    Net_ND2 = ND2.add_component(model=nicmodel, name="ND_B").get_interfaces()[0] 

    #Lan
    LN = slice.add_l2network(name="Lan", interfaces=[Net_ND1, Net_ND2])
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

### 1.2 Set up the Experiment Network
This section of the slice creation, once the nodes are active, will assign the network spaces some predefined values that we will use for the Assignment.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("10.0.0.0/24")
    Network = 'Lan'
    
    A = slice.get_node('ND_A')
    B = slice.get_node('ND_B')
    
    #A's lan
    A_iface = A.get_interface(network_name=Network)  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name=Network)  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
            if (node.get_name() == "ND_A"):
                A_Device_Name = interface.get_device_name()
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")
        
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any additional software, tools or scripts that we need for our experiments.

In [ ]:
# Install Software
try:
    INTERFACE_NAME_HOLDER = "INTERFACE_NAME"

    for node in slice.get_nodes():        
        node.execute("sudo apt update")
        node.execute("sudo apt install net-tools")
        node.execute("sudo apt install iperf")
        
        #upload scripts to make a graphic
        node.upload_file("./scripts/ss-data-analysis.R","ss-data-analysis.R")
        node.upload_file("./scripts/ss-output.sh","ss-output.sh")
        node.execute(f"sudo tc qdisc add dev {A_Device_Name} root tbf rate 13mbit limit 1000mb burst 10kb")
        node.execute(f"sed -i 's/{INTERFACE_NAME_HOLDER}/{A_Device_Name}/' ss-output.sh")
        node.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x ss-output.sh ")

except Exception as e:
    print(f"Fail: {e}")

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the TCP Traffic Notebook. You can either open it on the explorer or click [Here](./TCP_Traffic.ipynb) to open the next notebook.